In [ ]:
!pip install spacy
!pip install pickle

In [125]:
import spacy
from spacy.training.example import Example
import random
import pickle

In [126]:
train_data = pickle.load(open(r'data\training\train_data.pkl', 'rb'))
train_data[0]

('Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [ ]:
train_data[1]


In [128]:
nlp = spacy.blank("en")

if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner")
else:
    ner = nlp.get_pipe("ner")



In [129]:
for _, annotations in train_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])


In [133]:
from spacy.util import minibatch, compounding
import warnings

In [134]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()

    for epoch in range(20):  # You can tune this
        random.shuffle(train_data)
        losses = {}
        batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.5))

        for batch in batches:
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                try:
                    example = Example.from_dict(doc, annotations)
                    nlp.update([example], drop=0.3, losses=losses)
                except ValueError as e:
                    warnings.warn(f"Skipping example due to overlap: {e}")

        print(f"Epoch {epoch+1}: Losses {losses}")

c:\Users\ashis\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Gaikwad Dilip Microsoft Dainamic Software Billing ..." with entities "[(951, 990, 'Skills'), (926, 941, 'Companies worke...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\ashis\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Ravi Shankar Working as Escalation Engineer with M..." with entities "[(4016, 4025, 'Companies worked at'), (3941, 3950,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\Users\ashis\AppData\Local\Temp\ipykernel_16160\3944

Epoch 1: Losses {'ner': np.float32(8252.094)}


KeyboardInterrupt: 

In [132]:
nlp.to_disk("custom_ner_model")
print("Model saved to 'custom_ner_model'")

Model saved to 'custom_ner_model'


In [ ]:
nlp_model = spacy.load("custom_ner_model")

print("Entities Found:")
for ent in doc.ents:
    print(ent.text, ent.label_)